In [1]:
import numpy, scipy, matplotlib.pyplot as plt, pandas, librosa
from IPython.display import Audio
%matplotlib inline
import matplotlib
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import os
from os import path
import sys
import random
from random import randint

In [2]:
slicelength = 50
feature_power = 2
data_proportion = .01

classical_names = [f for f in os.listdir("./classical_short_%i/" % slicelength)]
country_names = [f for f in os.listdir("./country_short_%i/" % slicelength)]

num_datapoints = int(min(len(classical_names), len(country_names)) * data_proportion)
print "num datapoints: " + str(num_datapoints)

random.shuffle(classical_names)
random.shuffle(country_names)
classical_names = classical_names[0:num_datapoints]
country_names = country_names[0:num_datapoints]

def stft_array(names, kind):
    update_per = len(names)/10
    print "update per %i" % update_per
    array = []
    for i, x in enumerate(names):
        song_path = '%s_short_%i/%s' % (kind, slicelength, x)
        if i%update_per == 0:
            sys.stdout.write(str(i/update_per) + " ")
            print "adding " + song_path + " to " + kind
        csong_stft = librosa.stft(librosa.load(song_path)[0])
        csong_log_stft = librosa.logamplitude(np.abs(csong_stft)**feature_power,ref_power=np.max)
        csong_log_stft = [abs(item)/80.0 for sublist in csong_log_stft for item in sublist]
        array.append(csong_log_stft)
    return array
    
classical_train = stft_array(classical_names, 'classical')
country_train = stft_array(country_names, 'country')

test_classical = []
test_country = []

print "number of classical slices: %i" % len(classical_train)

while len(test_classical) < len(classical_train) / 10:
    r = randint(0, len(classical_train) - 1)
    test_classical.append(classical_train[r])
    del classical_train[r]
    
print "number of classical training slices: %i" % len(classical_train)
print "number of classical testing slices: %i" % len(test_classical)

print "number of country slices: %i" % len(country_train)
while len(test_country) < len(country_train) / 10:
    r = randint(0, len(country_train) - 1)
    test_country.append(country_train[r])
    del country_train[r]
    
print "number of country training slices: %i" % len(country_train)
print "number of country testing slices: %i" % len(test_country)

num datapoints: 408
update per 40
0 adding classical_short_50/song0_8041.mp3 to classical
1 adding classical_short_50/song9_5886.mp3 to classical
2 adding classical_short_50/song5_3762.mp3 to classical
3 adding classical_short_50/song4_2526.mp3 to classical
4 adding classical_short_50/song2_3554.mp3 to classical
5 adding classical_short_50/song4_890.mp3 to classical
6 adding classical_short_50/song6_2044.mp3 to classical
7 adding classical_short_50/song1_31.mp3 to classical
8 adding classical_short_50/song3_270.mp3 to classical
9 adding classical_short_50/song0_15537.mp3 to classical
10 adding classical_short_50/song1_1924.mp3 to classical
update per 40
0 adding country_short_50/song7_4008.mp3 to country
1 adding country_short_50/song6_1190.mp3 to country
2 adding country_short_50/song5_457.mp3 to country
3 adding country_short_50/song6_2941.mp3 to country
4 adding country_short_50/song8_783.mp3 to country
5 adding country_short_50/song6_50.mp3 to country
6 adding country_short_50/song

In [3]:
testing_with_mnist = False

if testing_with_mnist:
    # testing to see if mnist will work here
    from tensorflow.examples.tutorials.mnist import input_data
    mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

    training = mnist.train.images
    testing = mnist.test.images
    training_validation_one_hot = mnist.train.labels
    testing_validation_one_hot = mnist.test.labels
else:
    training = (numpy.array(classical_train + country_train)).astype(numpy.float32)
    testing = (numpy.array(test_classical + test_country)).astype(numpy.float32)
#     tr_len = len(temp_train)
#     te_len = len(temp_testing)
#     in_len = len(temp_train[0])

#     training = [[temp_training[i][j] for j in range(in_len)] for i in range(tr_len)]
#     testing = [[temp_testing[i][j] for j in range(in_len)] for i in range(te_len)]
    
#     for i in range(100):
#         for j in range(10):
#             print type(training[i][j])
#             print training[i][j]
#             print type(mnist.train.images[i][j])    
#             print mnist.train.images[i][j]

    classical_one_hot = numpy.array([0,1])
    country_one_hot = numpy.array([1,0])
    training_validation_one_hot = (numpy.array([classical_one_hot for x in classical_train] + [country_one_hot for x in country_train])).astype(numpy.float32)
    testing_validation_one_hot = (numpy.array([classical_one_hot for x in test_classical] + [country_one_hot for x in test_country])).astype(numpy.float32)

def type_check(array_types, num_types):
    error_flag = False
    for x in array_types:
        if type(x) is numpy.ndarray:
            print "ARRAY TYPE ERROR"
            error_flag = True
    for x in num_types:
        if type(x) is numpy.float32:
            print "NUM TYPE ERROR"
            error_flag = True 
    if not error_flag:
        print "All types check out"
        
array_types = [type(training), type(training[0]), type(training_validation_one_hot), type(training_validation_one_hot[0]),
              type(testing), type(testing[0]), type(testing_validation_one_hot), type(testing_validation_one_hot[0])]
num_types = [type(training[0][0]), type(training_validation_one_hot[0][0]), type(testing[0][0]), type(testing_validation_one_hot[0][0])]
type_check(array_types, num_types)      

if len(training)==len(training_validation_one_hot) and len(testing) == len(testing_validation_one_hot) and len(training[0])==len(testing[0]):
    print "training: " + str(len(training))
    print "testing: " + str(len(testing))
    print "num features: " + str(len(training[0]))
else:
    print "ERROR"

All types check out
training: 742
testing: 74
num features: 4100


In [4]:
# max_array = [max(x) for x in training]
# print "max val: " + str(max(max_array))
# min_array = [min(x) for x in training]
# print "min val: " + str(min(min_array))
# print "peek: " + str(training[10][0:200])

In [5]:
def shuffle_array(xs, ys):
    zipped = zip(xs,ys)
    random.shuffle(zipped)
    return zip(*zipped)
print training_validation_one_hot[0:10]
type_check(array_types, num_types)

training, training_validation_one_hot = shuffle_array(training, training_validation_one_hot)
testing, testing_validation_one_hot = shuffle_array(testing, testing_validation_one_hot)

print training_validation_one_hot[0:10]
type_check(array_types, num_types)

print training[5:10]

[[ 0.  1.]
 [ 0.  1.]
 [ 0.  1.]
 [ 0.  1.]
 [ 0.  1.]
 [ 0.  1.]
 [ 0.  1.]
 [ 0.  1.]
 [ 0.  1.]
 [ 0.  1.]]
All types check out
(array([ 1.,  0.], dtype=float32), array([ 0.,  1.], dtype=float32), array([ 1.,  0.], dtype=float32), array([ 0.,  1.], dtype=float32), array([ 1.,  0.], dtype=float32), array([ 1.,  0.], dtype=float32), array([ 1.,  0.], dtype=float32), array([ 1.,  0.], dtype=float32), array([ 0.,  1.], dtype=float32), array([ 0.,  1.], dtype=float32))
All types check out
(array([ 0.18023482,  0.24476567,  0.29893789, ...,  1.        ,
        1.        ,  1.        ], dtype=float32), array([ 0.39146048,  0.32137173,  0.35087502, ...,  1.        ,
        1.        ,  1.        ], dtype=float32), array([ 0.29891086,  0.35027358,  0.42402467, ...,  1.        ,
        1.        ,  1.        ], dtype=float32), array([ 0.16271943,  0.21619442,  0.41105899, ...,  1.        ,
        1.        ,  1.        ], dtype=float32), array([ 0.36539441,  0.39835516,  0.47473502, ..., 

In [6]:
numLabels = len(training_validation_one_hot[0])
featureSize = len(training[0])
print "num labels: "+str(numLabels)
print "num features: " + str(featureSize)

x = tf.placeholder(tf.float32, [None, featureSize])
W = tf.Variable(tf.zeros([featureSize, numLabels]))
b = tf.Variable(tf.zeros([numLabels]))

y = tf.nn.softmax(tf.matmul(x,W) + b)

y_ = tf.placeholder(tf.float32, [None, numLabels])
cross_entropy = tf.reduce_mean(-tf.reduce_sum(y_ * tf.log(y), reduction_indices=[1]))

train_step = tf.train.GradientDescentOptimizer(0.01).minimize(cross_entropy)

init = tf.initialize_all_variables()
sess = tf.InteractiveSession()
sess.run(init)

# def weight_variable(shape):
#     initial = tf.truncated_normal(shape, stddev=0.1)
#     return tf.Variable(initial)

# def bias_variable(shape):
#     initial = tf.constant(0.1, shape=shape)
#     return tf.Variable(initial)

# #convolution and pooling (may not need this)
# def conv2d(x, W):
#     return tf.nn.conv2d(x, W, strides=[1,1,1,1], padding='SAME')
# def max_pool_2x2(x):
#     return tf.nn.max_pool(x, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')

# #convolution layer
# W_conv1 = weight_variable([5, 5, 1, 32])
# b_conv1 = bias_variable([32])

# x_image = tf.reshape(x, [-1, 28, 28, 1])

num labels: 2
num features: 4100


In [7]:
training_length = len(training)
batch_size = 3
num_iters = training_length / batch_size


print "num iters: " + str(num_iters)

for i in range(num_iters):
    if i%(num_iters/100) == 0:
        sys.stdout.write(str(i/(num_iters/100)) + " ")
    start = i*batch_size
    end = (i+1)*batch_size
    batch_xs = training[start:end]
    batch_ys = training_validation_one_hot[start:end]
    sess.run(train_step, feed_dict={x: batch_xs, y_: batch_ys})
print "\nTraining Complete"

testing = numpy.asarray(testing)
feed_dict = {x: testing}
correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(y_, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
print (sess.run(accuracy, feed_dict={x: testing, y_: testing_validation_one_hot}))

classification = sess.run(y, feed_dict)
print classification[0:10]
print classification.shape
print testing.shape
for i in range(10):
    test = numpy.asarray(classification[i])
    label = numpy.asarray(testing_validation_one_hot[i])
    print str(test.argmax()) + " " + str(label.argmax())

num iters: 247
0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 100 101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 
Training Complete
0.513514
[[  9.76127684e-01   2.38722786e-02]
 [  9.88279581e-01   1.17203891e-02]
 [  9.91206706e-01   8.79326090e-03]
 [  9.98010099e-01   1.98990945e-03]
 [  7.55190372e-01   2.44809657e-01]
 [  9.46132898e-01   5.38670905e-02]
 [  9.44687843e-01   5.53121753e-02]
 [  9.58514333e-01   4.14856263e-02]
 [  9.99992609e-01   7.41777103e-06]
 [  9.94619489e-01   5.38050383e-03]]
(74, 2)
(74, 4100)
0 1
0 1
0 1
0 1
0 1
0 1
0 1
0 0
0 0
0 1


In [8]:
print W